# Comparing linear rate model with linear Hawkes, steady state rates at a fixed h, then for varying h

In [ ]:
# Installation
using Revise
using LinearAlgebra,Statistics,StatsBase,Distributions
using Plots,NamedColors ; theme(:default)
using FFTW
using ProgressMeter
using Random
Random.seed!(0)
using HawkesSimulator; const global H = HawkesSimulator

In [ ]:
# Initializing variables
# one E, one I population
v_rest = 0
alpha = 0.3
n = 1
tau = [20.0, 10.0]*1E-3   #time constant, [ E, I ]
tau_noise = 50*1E-3
w = [1.25 -0.65
     1.2 -0.5]
w_hawkes = w .* alpha
time_step = 1*1E-3
noise = [0.0 0.0] #ignoring

In [ ]:
function rate_powerlaw(v::Float64)
    diff = v - v_rest
    diff = (diff < 0) ? 0 : diff
    return alpha*(diff^n)
end

function plot_powerlaw()
    count = 81
    v_arr = zeros(count)
    v_arr[1] = -80
    for i in 2:count
        v_arr[i] = v_arr[i-1] +1
    end
    rates = rate_powerlaw.(v_arr)
    plot(v_arr, rates, xlabel="Voltage", ylabel="Rate")
end

# plot_powerlaw()

In [ ]:
function onedmat(x::Real)
  return cat(x;dims=2)
end

In [ ]:
# Euler Method
function dv_ssn(v::Float64, h_i::Float64, noise_i::Float64, tau_i::Float64, w_arr::Vector{Float64}, rate_arr::Vector{Float64})
    num_neurons = size(w_arr, 1)
    dv = v_rest - v + h_i + noise_i
    for i in 1:num_neurons
        dv += w_arr[i]*rate_arr[i]
    end
    return dv*time_step/tau_i
end

In [ ]:
function simulate_ssn!(h, num_steps, t_arr, v_excite, v_inhibit, rate)
    
    v_excite[1] = v_rest
    v_inhibit[1] = v_rest
    rate[1,:] = [rate_powerlaw(v_excite[1]), rate_powerlaw(v_inhibit[1])]
    noise1 = zeros(2)
    for i in 2:num_steps
        t_arr[i] = t_arr[i-1] + time_step
        v_excite[i] = v_excite[i-1] + dv_ssn(v_excite[i-1], h, noise1[1], tau[1], [w[1,1], w[1,2]], rate[i-1,:])
        v_inhibit[i] = v_inhibit[i-1] + dv_ssn(v_inhibit[i-1], h, noise1[2], tau[2], [w[2,1], w[2,2]], rate[i-1,:])
        rate[i,:] = [rate_powerlaw(v_excite[i]), rate_powerlaw(v_inhibit[i])]
    end
    
end

In [ ]:
function single_h_linear_rate_simulation()
    h = 4.0
    t_max = 1
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    v_excite = zeros(num_steps)
    v_inhibit = zeros(num_steps)
    rate = zeros(num_steps, 2)
    simulate_ssn!(h, num_steps, t_arr, v_excite, v_inhibit, rate)
    return t_arr, v_excite, v_inhibit, rate
end

In [ ]:
t_arr, v_excite, v_inhibit, rate_linear = single_h_linear_rate_simulation()
println(size(t_arr))

In [ ]:
plt = plot(xlabel="time (s)", ylabel="V (mV)", title = "Linear Rate Model", fmt = :png, legend=:bottomright)
plot!(plt, t_arr , v_excite, label = "V_Excite", color="blue")
plot!(plt, t_arr , v_inhibit, label = "V_Inhibit", color="red")

Comparing with linear hawkes

In [ ]:
function simulate_hawkes!(networks, num_spikes)
    t_now = 0.0
    H.reset!.(networks) # clear spike trains etc
    for k in 1:num_spikes
        t_now = H.dynamics_step!(t_now, networks)
        if k%1_000 == 0
            H.clear_trains!(networks[1].postpops)
            H.clear_trains!(networks[2].postpops)
        end # clearing trains after every 1000 spikes
    end
    return t_now
end

In [ ]:
function single_h_linear_hawkes_simulation()
    h = 4.0
    n_spikes = 500_000
    baseline_rate_e = [alpha*h]
    baseline_rate_i = [alpha*h]
    tau_E = 20.0
    tau_I = 10.0
    pop_E = H.PopulationExp(tau_E, H.NLIdentity())
    pop_I = H.PopulationExp(tau_I, H.NLIdentity())
    popstate_E = H.PopulationState(pop_E,baseline_rate_e)
    popstate_I = H.PopulationState(pop_I, baseline_rate_i)
    network_E = H.InputNetwork(popstate_E,[popstate_E, popstate_I],[onedmat(w_hawkes[1,1]),onedmat(w_hawkes[1,2])])
    network_I = H.InputNetwork(popstate_I,[popstate_E, popstate_I],[onedmat(w_hawkes[2,1]),onedmat(w_hawkes[2,2])])
    simulate_hawkes!([network_E, network_I],n_spikes)
    rate = [H.numerical_rate(popstate_E.trains_history[1][2000:end]), H.numerical_rate(popstate_I.trains_history[1][2000:end])]
    return rate
end

In [ ]:
rate_hawkes = single_h_linear_hawkes_simulation()
println(rate_linear[size(rate_linear,1),:])
println(rate_hawkes)

In [ ]:
# plot for Rate vs time
length = size(t_arr,1)
plt = plot(xlabel="time (s)", ylabel="Rate (Hz)", fmt = :png, legend=:bottomright)
plot!(plt, t_arr[1:length] , rate_linear[1:length,1], label = "R_Linear_Excite", color="blue")
plot!(plt, t_arr[1:length] , rate_linear[1:length,2], label = "R_Linear_Inhibit",color="red")
r_excite_hawkes = zeros(length)
r_inhibit_hawkes = zeros(length)
for i in 1:length
    r_excite_hawkes[i] = rate_hawkes[1]
    r_inhibit_hawkes[i] = rate_hawkes[2]
end
plot!(plt, t_arr[1:length], r_excite_hawkes, label = "R_Hawkes_Excite",linestyle=:dash,linewidth=2, color="dark blue")
plot!(plt, t_arr[1:length], r_inhibit_hawkes, label = "R_Hawkes_Inhibit",linestyle=:dash,linewidth=2, color="dark red")

In [ ]:
using NamedColors
col_exc = colorant"dark blue"
col_exc2 = colorant"blue"
col_inh = colorant"dark red"
col_inh2 = colorant"red"

Comparing rates at multiple h values

In [ ]:
function run_ssn_multiple_h!(num_h, h_arr, rates_ssn)
    t_max = 1
    num_steps = Int(ceil(t_max/time_step))
    t_arr = zeros(num_steps)
    v_excite = zeros(num_steps)
    v_inhibit = zeros(num_steps)
    rate = zeros(num_steps, 2)
    @showprogress 1.0 "Running SSN multiple h..." for i in 1:num_h
        simulate_ssn!(h_arr[i], num_steps, t_arr, v_excite, v_inhibit, rate)
        rates_ssn[i,:] = rate[num_steps,:]
    end
end

In [ ]:
function run_hawkes_multiple_h!(num_h, h_arr, rates_hawkes, pop_E, pop_I)
    n_spikes = 500_000
    @showprogress 1.0 "Running Hawkes multiple h..." for i in 1:num_h
        baseline_rate_e = [alpha*(h_arr[i])]
        baseline_rate_i = [alpha*(h_arr[i])]
        popstate_E = H.PopulationState(pop_E,baseline_rate_e)
        popstate_I = H.PopulationState(pop_I, baseline_rate_i)
        network_E = H.InputNetwork(popstate_E,[popstate_E, popstate_I],[onedmat(w_hawkes[1,1]),onedmat(w_hawkes[1,2])])
        network_I = H.InputNetwork(popstate_I,[popstate_E, popstate_I],[onedmat(w_hawkes[2,1]),onedmat(w_hawkes[2,2])])
        simulate_hawkes!([network_E, network_I],n_spikes)
        rates_hawkes[i,:] = [H.numerical_rate(popstate_E.trains_history[1][2000:end]), H.numerical_rate(popstate_I.trains_history[1][2000:end])]
    end
end

In [ ]:
h_max = 15.0
h_min = 1.0
step = 1
num_h = Int(ceil((h_max-h_min+1)/step))
h_arr = zeros(num_h)
h_arr[1] = h_min
rates_linear = zeros(num_h,2)
rates_hawkes_linear = zeros(num_h,2)

function compare()
    for i in 2:num_h
        h_arr[i] = h_arr[i-1] + step
    end
    
    tau_E = 20.0
    tau_I = 10.0
    pop_E = H.PopulationExp(tau_E, H.NLIdentity())
    pop_I = H.PopulationExp(tau_I, H.NLIdentity())
    run_hawkes_multiple_h!(num_h, h_arr, rates_hawkes_linear, pop_E, pop_I)
    run_ssn_multiple_h!(num_h, h_arr, rates_linear)
end

In [ ]:
compare()

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="Excitatory Rate (Hz)", title="Linear Rate Model vs Linear Hawkes", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , rates_linear[:,1], label = "linear"; color=col_exc2)
plot!(plt, h_arr , rates_hawkes_linear[:,1], label = "hawkes"; color=col_exc)

In [ ]:
plt = plot(xlabel="h (mV)", ylabel="Inhibitory Rate (Hz)", title="Linear Rate Model vs Linear Hawkes", fmt = :png, legend=:bottomright)
plot!(plt, h_arr , rates_linear[:,2], label = "linear"; color=col_inh2)
plot!(plt, h_arr , rates_hawkes_linear[:,2], label = "hawkes"; color=col_inh)

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = 2*abs(rates_hawkes_linear[i,1] - rates_linear[i,1])/(rates_hawkes_linear[i,1] + rates_linear[i,1])
    y_i[i] = 2*abs(rates_hawkes_linear[i,2] - rates_linear[i,2])/(rates_hawkes_linear[i,2] + rates_linear[i,2])
end
plt = plot(xlabel = "h", ylabel="relative error", fmt = :png, legend=:bottomright)
plot!(plt, h_arr, y_e, label="rateE", color="blue")
plot!(plt, h_arr, y_i, label="rateI", color="red")

In [ ]:
y_e = zeros(num_h)
y_i = zeros(num_h)
for i in 2:num_h
    y_e[i] = abs(rates_hawkes_linear[i,1] - rates_linear[i,1])
    y_i[i] = abs(rates_hawkes_linear[i,2] - rates_linear[i,2])
end
plt = plot(xlabel = "h", ylabel="absolute error", fmt = :png, legend=:bottomright)
plot!(plt, h_arr, y_e, label="rateE", color="blue")
plot!(plt, h_arr, y_i, label="rateI", color="red")